# Schlesinger system reduction mod $p$

In [8]:
import numpy as np
import sympy

We will begin by attempting to construct the vector of rational functions given by $O_n(z) y_p(z)$, for a fixed $n$, but having $p = (p_1, \dots, p_m)$ as free parameters. See the notes for the meaning of these expressions.

In [130]:
n = 3 # Power to which we are raising the Picard-Fuchs differential operator
m = 2 # Dimension of the m \times m matrices in the Picard-Fuchs operator

# Matrices with integer entries
matrices = [
    sympy.Matrix([[1, 1], [1, 1]]),
    sympy.Matrix([[1, 2], [4, -1]])
]

# (a_j, b_j) integer pairs
a = [1, 2]
b = [2, -1]

# Creates the symbols corresponding to the p_j, and z
p = sympy.symbols(" ".join(["p_{}".format(j+1) for j in range(m)]), integer=True)
z = sympy.Symbol("z")

# Creates y_p(z)
y = sympy.Matrix([z ** p_j for p_j in p])

# Creates A(z)
A = None

for j, M in enumerate(matrices):
    M_prime = (1/(a[j] * z + b[j])) * M
    if A is None:
        A = M_prime
    else:
        A = A + M_prime

sympy.cancel(A, z)

# Creates the differential operator d/dz - A(z)
def D_op(y):
    expr = sympy.cancel(sympy.diff(y, z) - A @ y, z)
    return expr

# Creates O_n
def O(n):
    def fn(z):
        for _ in range(n):
            z = D_op(z)
        return z
    return fn

O_n = O(n)

In [138]:
# From here, we can iterate D_op
R = O_n(y)

In [139]:
R

Matrix([
[(8*p_1**3*z**6*z**p_1 + 36*p_1**3*z**5*z**p_1 + 30*p_1**3*z**4*z**p_1 - 45*p_1**3*z**3*z**p_1 - 30*p_1**3*z**2*z**p_1 + 36*p_1**3*z*z**p_1 - 8*p_1**3*z**p_1 - 60*p_1**2*z**6*z**p_1 - 228*p_1**2*z**5*z**p_1 - 135*p_1**2*z**4*z**p_1 + 240*p_1**2*z**3*z**p_1 + 90*p_1**2*z**2*z**p_1 - 120*p_1**2*z*z**p_1 + 24*p_1**2*z**p_1 + 286*p_1*z**6*z**p_1 + 879*p_1*z**5*z**p_1 + 561*p_1*z**4*z**p_1 - 252*p_1*z**3*z**p_1 - 246*p_1*z**2*z**p_1 + 84*p_1*z*z**p_1 - 16*p_1*z**p_1 - 48*p_2**2*z**6*z**p_2 - 180*p_2**2*z**5*z**p_2 - 120*p_2**2*z**4*z**p_2 + 135*p_2**2*z**3*z**p_2 + 60*p_2**2*z**2*z**p_2 - 36*p_2**2*z*z**p_2 + 192*p_2*z**6*z**p_2 + 492*p_2*z**5*z**p_2 + 186*p_2*z**4*z**p_2 - 132*p_2*z**3*z**p_2 - 126*p_2*z**2*z**p_2 + 36*p_2*z*z**p_2 - 417*z**6*z**p_1 - 276*z**6*z**p_2 - 827*z**5*z**p_1 - 361*z**5*z**p_2 - 929*z**4*z**p_1 - 457*z**4*z**p_2 - 386*z**3*z**p_1 - 178*z**3*z**p_2)/(8*z**9 + 36*z**8 + 30*z**7 - 45*z**6 - 30*z**5 + 36*z**4 - 8*z**3)],
[  (-72*p_1**2*z**6*z**p_1 - 300*p_1**

In [152]:
# We can then simplify and extract the numerator and denominator of each entry
num_list, denom_list = [], []

for v in range(m):
    # We will record the "coefficient" of each z^{p_j}, in the numerators
    p_j_num_list, p_j_denom_list = [], []
    for j in range(m):
        p_j_num_list.append(sympy.collect(sympy.numer(R[v]).coeff(z ** p[j]), z))
    denom_list.append(sympy.collect(sympy.denom(R[v]), z))
    
    num_list.append(p_j_num_list)
    denom_list.append(p_j_denom_list)

We now need to check that each of the corresponding polynomials of the $p_j$ are $0 \ \text{mod} \ p$. These polynomials will look like this, for example:

In [157]:
num_list[0][0]

-8*p_1**3 + 24*p_1**2 - 16*p_1 + z**6*(8*p_1**3 - 60*p_1**2 + 286*p_1 - 417) + z**5*(36*p_1**3 - 228*p_1**2 + 879*p_1 - 827) + z**4*(30*p_1**3 - 135*p_1**2 + 561*p_1 - 929) + z**3*(-45*p_1**3 + 240*p_1**2 - 252*p_1 - 386) + z**2*(-30*p_1**3 + 90*p_1**2 - 246*p_1) + z*(36*p_1**3 - 120*p_1**2 + 84*p_1)

In particular, each power of $z$ corresponds to a different polynomial in $p_1$ or $p_2$

It seems like there is an easy way to do this: https://math.stackexchange.com/questions/4672283/polynomials-that-are-zero-as-functions-on-finite-fields